In [32]:
import os
import firebase_admin
from firebase_admin import credentials, db, firestore
import pandas as pd
from flask import Flask, request, jsonify

In [69]:
# ✅ Get Firebase credentials path
cred_path = os.getenv("FIREBASE_KEY_PATH")

if not cred_path:
    raise ValueError("FIREBASE_KEY_PATH is not set. Make sure the environment variable is configured.")

# ✅ Define database URL
DATABASE_URL = "https://umsports-linksync-default-rtdb.europe-west1.firebasedatabase.app/"

# ✅ Print debug info
print("🔍 Using Firebase Key Path:", cred_path)
print("🔍 Using Database URL:", DATABASE_URL)

# ✅ Initialize Firebase if not already initialized
if not firebase_admin._apps:
    cred = credentials.Certificate(cred_path)
    firebase_admin.initialize_app(cred, {'databaseURL': DATABASE_URL})

# ✅ Check if databaseURL is set correctly
app = firebase_admin.get_app()
print("✅ Firebase initialized with options:", app.options)

# ✅ Explicitly print databaseURL
print("🔍 Confirmed databaseURL in Firebase app:", app.options.get("databaseURL"))

# ✅ Connect to Realtime Database
ref = db.reference("/")  # Root of the database

# ✅ Test writing data
ref.child("test").set({
    "message": "Realtime Database is working!"
})

print("✅ Realtime Database connection successful! Check Firebase Console for 'test' data.")
app = Flask(__name__)

🔍 Using Firebase Key Path: C:\Users\maria\Documents\umsports-linksync-firebase-adminsdk-fbsvc-11d77d7d69.json
🔍 Using Database URL: https://umsports-linksync-default-rtdb.europe-west1.firebasedatabase.app/
✅ Firebase initialized with options: <firebase_admin._AppOptions object at 0x0000020D1CD3EC30>
🔍 Confirmed databaseURL in Firebase app: https://umsports-linksync-default-rtdb.europe-west1.firebasedatabase.app/
✅ Realtime Database connection successful! Check Firebase Console for 'test' data.


In [70]:
#check comment
import random
def generate_random_data():
    return {
        "skill_level": random.choice(["Beginner", "Intermediate", "Advanced"]),
        "weight_class": random.choice(["Lightweight", "Middleweight", "Heavyweight"]),
        "height": random.randint(150, 200),  # Height in cm
        "training_frequency": random.randint(1, 5),  # Days per week
        "preferred_training_time": random.choice(["Morning", "Afternoon", "Evening"]),
        "fighting_style": random.choice(["Speed", "Power", "Defense", "Technical", "Balanced"]),
        "previous_experience": random.choice(["None", "Boxing", "Muay Thai", "Karate", "Wrestling"]),
        "injury_history": random.choice(["None", "Minor Injuries", "Past Fracture", "Joint Issues"]),
        "preferred_cross_training": random.choice([
            ["Strength Training", "Yoga"],
            ["Cardio", "Agility Drills"],
            ["Endurance Running", "Flexibility Drills"]
        ])
    }

def upload_users_to_firebase():
    file_path = "users.xlsx"  # Ensure this file is in the correct path
    users_df = pd.read_excel(file_path)

    users_ref = db.reference("users")  # Reference to Firebase "users" node

    for _, row in users_df.iterrows():
        username = row.get("Username", f"user_{random.randint(1000,9999)}")
        password = row.get("Password", "default_pass")
        name = row.get("Name", "Anonymous")

        user_data = generate_random_data()

        # Overwrite generated values with actual ones from the file if available
        user_data["skill_level"] = row.get("Skill Level", user_data["skill_level"])
        user_data["weight_class"] = row.get("Weight Class", user_data["weight_class"])
        user_data["height"] = row.get("Height", user_data["height"])
        user_data["training_frequency"] = row.get("Training Frequency", user_data["training_frequency"])
        user_data["preferred_training_time"] = row.get("Preferred Training Time", user_data["preferred_training_time"])
        user_data["fighting_style"] = row.get("Fighting Style Preference", user_data["fighting_style"])
        user_data["previous_experience"] = row.get("Previous Experience", user_data["previous_experience"])
        user_data["injury_history"] = row.get("Injury History", user_data["injury_history"])
        user_data["preferred_cross_training"] = row.get("Preferred Cross-Training Activities", user_data["preferred_cross_training"])

        # Add user data to Firebase
        users_ref.child(username).set({
            "password": password,
            "name": name,
            **user_data  # Merges generated and actual data
        })

    print("✅ All users uploaded to Firebase successfully!")

# Run the function to upload users
upload_users_to_firebase()

✅ All users uploaded to Firebase successfully!


In [71]:
import numpy as np
from sklearn.cluster import KMeans
import time

# Skill level mapping
SKILL_LEVEL_MAP = {
    "Beginner": 1,
    "Intermediate": 2,
    "Advanced": 3
}

def cluster_users():
    users_ref = db.reference("users")
    users = users_ref.get()
    user_vectors = []
    user_ids = []

    for user_id, u_data in users.items():
        if not isinstance(u_data, dict):  # ✅ Ensure data is a dictionary
            print(f"Skipping invalid user data for {user_id}")
            continue

        # Convert skill level to numeric
        skill_level_str = str(u_data.get("skill_level", "Beginner")).strip()
        skill_level = SKILL_LEVEL_MAP.get(skill_level_str, 1)

        # Ensure training frequency is a number
        training_frequency = u_data.get("training_frequency", 1)
        if isinstance(training_frequency, str) and training_frequency.isdigit():
            training_frequency = int(training_frequency)
        elif not isinstance(training_frequency, (int, float)):
            training_frequency = 1  # Default value

        # Compute feedback score
        feedback_data = u_data.get("feedback", {})
        if feedback_data:
            try:
                feedback_scores = np.mean([
                    sum(feedback.values()) / len(feedback)
                    for feedback in feedback_data.values()
                ])
            except Exception:
                feedback_scores = 0  # Default if there's an issue
        else:
            feedback_scores = 0

        vector = [
            float(skill_level),  # ✅ Force float conversion
            float(training_frequency),  # ✅ Ensure numeric
            float(feedback_scores),  # ✅ Ensure numeric
            float(len(feedback_data)),  # ✅ Ensure numeric
        ]

        # Debugging print to ensure correct data format
        print(f"User {user_id} - Vector: {vector}")
        
        user_vectors.append(vector)
        user_ids.append(user_id)

    # **Ensure `user_cluster_map` is always defined**
    user_cluster_map = {}

    # Ensure KMeans runs only if there's more than one user
    if len(user_vectors) > 1:
        kmeans = KMeans(n_clusters=min(5, len(user_vectors)), random_state=42).fit(user_vectors)
        user_cluster_map = dict(zip(user_ids, kmeans.labels_))
    elif len(user_ids) == 1:  # ✅ Assign cluster 0 ONLY if there's exactly one user
        user_cluster_map[user_ids[0]] = 0
    else:
        print("⚠️ No users to cluster!")
        return  # Exit function early if no users exist
        
    # Store clusters in Firestore
    for user_id, cluster in user_cluster_map.items():
        cluster_int = int(cluster)
        print(f"User {user_id} → Cluster {cluster}")
        users_ref.child(user_id).update({"cluster": cluster_int})

    print("✅ User clustering updated.")

cluster_users()


User user1 - Vector: [1.0, 3.0, 0.0, 0.0]
User user10 - Vector: [1.0, 5.0, 0.0, 0.0]
User user11 - Vector: [2.0, 3.0, 0.0, 0.0]
User user12 - Vector: [2.0, 2.0, 0.0, 0.0]
User user13 - Vector: [2.0, 2.0, 0.0, 0.0]
User user14 - Vector: [1.0, 5.0, 0.0, 0.0]
User user15 - Vector: [1.0, 5.0, 0.0, 0.0]
User user16 - Vector: [1.0, 3.0, 0.0, 0.0]
User user17 - Vector: [2.0, 4.0, 0.0, 0.0]
User user18 - Vector: [3.0, 3.0, 0.0, 0.0]
User user19 - Vector: [1.0, 4.0, 0.0, 0.0]
User user2 - Vector: [2.0, 5.0, 0.0, 0.0]
User user20 - Vector: [3.0, 2.0, 0.0, 0.0]
User user21 - Vector: [1.0, 2.0, 0.0, 0.0]
User user22 - Vector: [2.0, 5.0, 0.0, 0.0]
User user23 - Vector: [3.0, 4.0, 0.0, 0.0]
User user24 - Vector: [2.0, 1.0, 0.0, 0.0]
User user25 - Vector: [2.0, 5.0, 0.0, 0.0]
User user26 - Vector: [2.0, 2.0, 0.0, 0.0]
User user27 - Vector: [2.0, 2.0, 0.0, 0.0]
User user28 - Vector: [3.0, 1.0, 0.0, 0.0]
User user29 - Vector: [1.0, 2.0, 0.0, 0.0]
User user3 - Vector: [2.0, 4.0, 0.0, 0.0]
User user30 - 

In [72]:
def on_feedback_change(event):
    """Triggers clustering when a user submits feedback."""
    print("🔄 Feedback updated. Re-clustering users...")
    cluster_users()

def on_session_change(event):
    """Triggers clustering when a user books a new session."""
    print("🔄 User booked a session. Re-clustering users...")
    cluster_users()

# Attach Firebase listeners for real-time updates
feedback_ref = db.reference("users")
feedback_watch = feedback_ref.listen(on_feedback_change)  # Watches user feedback

session_ref = db.reference("user_registrations")
session_watch = session_ref.listen(on_session_change)  # Watches session bookings

print("🚀 Listening for feedback and session changes...")

# Keep the script running to listen for Firebase changes
while True:
    time.sleep(10)

🔄 Feedback updated. Re-clustering users...🚀 Listening for feedback and session changes...

🔄 User booked a session. Re-clustering users...
User user1 - Vector: [1.0, 3.0, 0.0, 0.0]
User user10 - Vector: [1.0, 5.0, 0.0, 0.0]
User user11 - Vector: [2.0, 3.0, 0.0, 0.0]
User user12 - Vector: [2.0, 2.0, 0.0, 0.0]
User user13 - Vector: [2.0, 2.0, 0.0, 0.0]
User user14 - Vector: [1.0, 5.0, 0.0, 0.0]
User user15 - Vector: [1.0, 5.0, 0.0, 0.0]
User user16 - Vector: [1.0, 3.0, 0.0, 0.0]
User user17 - Vector: [2.0, 4.0, 0.0, 0.0]
User user18 - Vector: [3.0, 3.0, 0.0, 0.0]
User user19 - Vector: [1.0, 4.0, 0.0, 0.0]
User user2 - Vector: [2.0, 5.0, 0.0, 0.0]
User user20 - Vector: [3.0, 2.0, 0.0, 0.0]
User user21 - Vector: [1.0, 2.0, 0.0, 0.0]
User user22 - Vector: [2.0, 5.0, 0.0, 0.0]
User user23 - Vector: [3.0, 4.0, 0.0, 0.0]
User user24 - Vector: [2.0, 1.0, 0.0, 0.0]
User user25 - Vector: [2.0, 5.0, 0.0, 0.0]
User user26 - Vector: [2.0, 2.0, 0.0, 0.0]
User user27 - Vector: [2.0, 2.0, 0.0, 0.0]
Use

KeyboardInterrupt: 

In [53]:
def check_clusters():
    users_ref = db.reference("users")
    users = users_ref.get()

    if not users:
        print("No users found.")
        return

    missing_clusters = []

    print("🔹 Checking User Clusters in Firebase...")
    for user_id, u_data in users.items():
        cluster = u_data.get("cluster")
        if cluster is None:
            missing_clusters.append(user_id)
        print(f"User {user_id} → Cluster: {cluster}")

    if missing_clusters:
        print("⚠️ These users are missing clusters:", missing_clusters)
    else:
        print("✅ All users have clusters!")

# Run the check
check_clusters()


🔹 Checking User Clusters in Firebase...
User user1 → Cluster: 3
User user10 → Cluster: 4
User user11 → Cluster: 3
User user12 → Cluster: 4
User user13 → Cluster: 0
User user14 → Cluster: 1
User user15 → Cluster: 2
User user16 → Cluster: 1
User user17 → Cluster: 1
User user18 → Cluster: 0
User user19 → Cluster: 1
User user2 → Cluster: 4
User user20 → Cluster: 0
User user21 → Cluster: 1
User user22 → Cluster: 4
User user23 → Cluster: 0
User user24 → Cluster: 1
User user25 → Cluster: 1
User user26 → Cluster: 1
User user27 → Cluster: 0
User user28 → Cluster: 0
User user29 → Cluster: 4
User user3 → Cluster: 4
User user30 → Cluster: 2
User user31 → Cluster: 4
User user32 → Cluster: 1
User user33 → Cluster: 1
User user34 → Cluster: 3
User user35 → Cluster: 0
User user36 → Cluster: 1
User user37 → Cluster: 0
User user38 → Cluster: 0
User user39 → Cluster: 3
User user4 → Cluster: 2
User user40 → Cluster: 2
User user41 → Cluster: 0
User user42 → Cluster: 1
User user43 → Cluster: 0
User user44 → 

In [14]:
@app.route('/find_partner', methods=['POST'])
def find_partner():
    data = request.json
    user_id = data['user_id']
    session_id = data['session_id']
    personalized = data.get('personalized', False)

    user_cluster_map = cluster_users()

    if personalized:
        return personalized_partner_matching(user_id, session_id, user_cluster_map)
    else:
        return random_partner_matching(user_id, session_id)

# Random Sparring Partner Matching (30 minutes before session)
def random_partner_matching(user_id, session_id):
    available_partners = db.collection('registrations').where('session_id', '==', session_id).stream()
    partners_list = [partner for partner in available_partners if partner.id != user_id]

    if len(partners_list) >= 3:
        suggested_partners = random.sample(partners_list, 3)
    else:
        suggested_partners = partners_list

    return jsonify({'message': 'Choose a sparring partner', 'suggested_partners': [p.id for p in suggested_partners]}), 200

# Personalized Sparring Partner Matching
def personalized_partner_matching(user_id, session_id, user_cluster_map):
    user_cluster = user_cluster_map.get(user_id, None)
    matched_partners = []

    if user_cluster is not None:
        potential_partners = [u_id for u_id, cluster in user_cluster_map.items() if cluster == user_cluster and u_id != user_id]
        if len(potential_partners) >= 3:
            matched_partners = random.sample(potential_partners, 3)
        else:
            matched_partners = potential_partners

    return jsonify({'message': 'Personalized sparring partners assigned', 'suggested_partners': matched_partners}), 200

In [15]:
 # Collect Feedback After Sparring Session
@app.route('/submit_feedback', methods=['POST'])
def submit_feedback():
    data = request.json
    user_id = data['user_id']
    partner_id = data['partner_id']
    feedback = data['feedback']  # Dictionary containing answers to 10 feedback questions

    user_ref = db.collection('users').document(user_id)
    partner_ref = db.collection('users').document(partner_id)

    feedback_entry = {
        "partner_id": partner_id,
        "session_id": data.get("session_id"),
        "skill_match": feedback.get("skill_match"),
        "intensity": feedback.get("intensity"),
        "comfort": feedback.get("comfort"),
        "technique_rating": feedback.get("technique_rating"),
        "repeat_preference": feedback.get("repeat_preference"),
        "biggest_struggle": feedback.get("biggest_struggle"),
        "fatigue_level": feedback.get("fatigue_level"),
        "weakness_exposed": feedback.get("weakness_exposed"),
        "training_recommendation": feedback.get("training_recommendation"),
        "cross_training_interest": feedback.get("cross_training_interest"),
        "timestamp": firestore.SERVER_TIMESTAMP
    }

    # Store feedback in Firestore
    db.collection('users').document(user_id).collection('feedback').add(feedback_entry)
    db.collection('users').document(partner_id).collection('received_feedback').add(feedback_entry)

    return jsonify({'message': 'Feedback submitted successfully'}), 200

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\maria\IdeaProjects\test1\venv\Lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [73]:
import firebase_admin
from firebase_admin import credentials, db
import re
import time

# Reference to Firebase
ref = db.reference("/")

def move_pending_sessions(pending_data):

    if not pending_data:
        print("No pending registrations found.")
        return

    # Extract user and session details
    match = re.match(r"(\w+)\s(.+)\s-\s([\d:]+)\sto\s([\d:]+)\son\s([\d-]+)", pending_data)

    if match:
        user_id, sport, start_time, end_time, date = match.groups()

        # Define session data
        session_data = {
            "sport": sport,
            "start_time": start_time,
            "end_time": end_time,
            "date": date
        }

        # Update the user's session in `user_registrations`
        user_sessions_ref = ref.child(f"user_registrations/{user_id}/sessions")
        user_sessions_ref.push(session_data)

        # Remove the entry from `pending_registrations`
        ref.child("pending_registrations").delete()

        print(f"✅ Moved session for {user_id} to user_registrations and removed from pending_registrations.")
    else:
        print("❌ Failed to parse session data.")

# Function to listen for changes in Firebase
def listen_for_pending_registrations(event):
    """Listens for new pending registrations and moves them automatically."""
    print("🔄 Change detected in pending_registrations!")
    pending_data = event.data
    move_pending_sessions(pending_data)

# Attach a listener to detect changes in `pending_registrations`
pending_ref = ref.child("pending_registrations")
pending_ref.listen(listen_for_pending_registrations)

print("🚀 Listening for new pending registrations...")

# Keep script running
while True:
    time.sleep(10)


🚀 Listening for new pending registrations...
🔄 Change detected in pending_registrations!
✅ Moved session for user51 to user_registrations and removed from pending_registrations.
🔄 Change detected in pending_registrations!
No pending registrations found.


KeyboardInterrupt: 

In [84]:
def get_logged_in_user():
    """Retrieve the currently logged-in user from Firebase."""
    login_ref = db.reference("login_status/user")  # Fetch the logged-in user
    logged_in_user = login_ref.get()

    if not logged_in_user:
        print("⚠️ No user is currently logged in.")
        return None

    print(f"✅ Logged in user detected: {logged_in_user}")
    return logged_in_user

def store_user_sessions_temp():
    """Fetch logged-in user's sessions and store them temporarily in Firebase."""

    logged_in_user = get_logged_in_user()  # 🔹 Automatically detect user
    if not logged_in_user:
        return

    # Fetch the user's registered sessions
    user_sessions_ref = db.reference(f"user_registrations/{logged_in_user}/sessions")
    user_sessions = user_sessions_ref.get()

    if not user_sessions:
        print(f"⚠️ No sessions found for {logged_in_user}")
        return
    

    #  🔹 Extract session codes and sport names
    session_dict = {}  # Dictionary to store session_code: sport_name
    sport_names_list = []  # Store only sport names + dates

    # 🔹 Handle both dictionary and list cases
    if isinstance(user_sessions, dict):
        session_items = user_sessions.items()  # ✅ Extract items if it's a dictionary
    elif isinstance(user_sessions, list):
        session_items = enumerate(user_sessions)  # ✅ Assign index as session_code for lists
    else:
        print("⚠️ Unexpected data format in Firebase!")
        return

    for session_code, session_info in session_items:
        if isinstance(session_info, dict):  # ✅ Correctly extract sport and date
            sport_name = session_info.get("sport", "Unknown Sport").strip()
            session_date = session_info.get("date", "Unknown Date")

            display_text = f"{sport_name} - {session_date}"  # Format for Thunkable
            session_dict[str(session_code)] = display_text  # Store session_code: display_text mapping
            sport_names_list.append(display_text)  # Store only names + dates for display
        else:
            print(f"⚠️ Skipping invalid session entry: {session_info}")

    if not sport_names_list:
        print(f"⚠️ No valid sessions found for {logged_in_user}")
        return

    # 🔹 Store session names as a single string in Firebase for Thunkable
    sport_names_string = " | ".join(sport_names_list)  # Example: "Kickboxing | Boxing | Strength Training"
    temp_ref = db.reference("temp_sessions")  # ✅ No user hierarchy
    temp_ref.set(sport_names_string)

    # 🔹 Store session code mapping for backend use
    session_mapping_ref = db.reference("session_mappings")
    session_mapping_ref.set(session_dict)  # ✅ Store session codes separately

    print(f"✅ Sport names stored for {logged_in_user}: {sport_names_string}")
    print(f"✅ Session code mapping stored: {session_dict}")


# 🔹 Run the function automatically when script executes
store_user_sessions_temp()

✅ Logged in user detected: user51
⚠️ Skipping invalid session entry: Test Session 51
✅ Sport names stored for user51: Kickboxing - 2025-02-18
✅ Session code mapping stored: {'-OJocedaXLEjhzbWJXOn': 'Kickboxing - 2025-02-18'}


In [83]:
# Define allowed sports
ALLOWED_SPORTS = {"Kickboxing", "Boxing", "Self Defense"}

def get_logged_in_user():
    """Retrieve the currently logged-in user from Firebase."""
    login_ref = db.reference("login_status/user")  # Fetch the logged-in user
    logged_in_user = login_ref.get()

    if not logged_in_user:
        print("⚠️ No user is currently logged in.")
        return None

    print(f"✅ Logged in user detected: {logged_in_user}")
    return logged_in_user

def store_filtered_sessions_temp():
    """Fetch user's registered sessions, filter by allowed sports, and store them as a string in Firebase."""

    logged_in_user = get_logged_in_user()  # 🔹 Automatically detect user
    if not logged_in_user:
        return

    # Fetch the user's registered sessions
    user_sessions_ref = db.reference(f"user_registrations/{logged_in_user}/sessions")
    user_sessions = user_sessions_ref.get()

    if not user_sessions:
        print(f"⚠️ No sessions found for {logged_in_user}")
        return

    # 🔹 Extract only Kickboxing, Boxing, and Self Defense sessions
    
    filtered_sport_names_list = []  # Store only sport names + dates for display
    session_items = []
    # 🔹 Handle both dictionary and list cases
    if isinstance(user_sessions, dict):
        session_items = user_sessions.items()  # ✅ If dictionary, extract values
    elif isinstance(user_sessions, list):
        session_items = enumerate(user_sessions)  # ✅ If list, use directly
    else:
        print("⚠️ Unexpected data format in Firebase!")
        return


    for session_code, session_info in session_items:
        if isinstance(session_info, dict):  # ✅ Correctly extract sport and date
            sport_name = session_info.get("sport", "Unknown Sport").strip()
            session_date = session_info.get("date", "Unknown Date")

            if sport_name in ALLOWED_SPORTS:  # ✅ Filter by allowed sports
                display_text = f"{sport_name} - {session_date}"  # Format for Thunkable
                filtered_sport_names_list.append(display_text)  # Store only names + dates for display

        else:
            print(f"⚠️ Skipping invalid session entry: {session_info}")

    if not filtered_sport_names_list:
        print(f"⚠️ No fight sessions found for {logged_in_user}")
        return


    # 🔹 Store filtered session names + dates as a single string in Firebase
    filtered_sport_names_string = " | ".join(filtered_sport_names_list)  # Example: "Kickboxing - 2025-02-17 | Boxing - 2025-02-18"
    temp_ref = db.reference("temp_filtered_sessions")  # ✅ No user hierarchy
    temp_ref.set(filtered_sport_names_string)  # ✅ Store as a direct entry

    print(f"✅ Filtered sport names stored for {logged_in_user}: {filtered_sport_names_string}")

# 🔹 Run the function automatically when script executes
store_filtered_sessions_temp()

✅ Logged in user detected: user51
⚠️ Skipping invalid session entry: Test Session 51
✅ Filtered sport names stored for user51: Kickboxing - 2025-02-18
